In [3]:
!python neat_test.py


[***]	Fitness: [720.91]


In [11]:
from neat_src.ann import getNodeOrder

import numpy as np

nodeG = np.array([
    [5, 7, 9],     # Node IDs (using distinct numbers)
    [1, 2, 4],     # Node types: input, output, bias
    [1, 1, 1]      # Activation functions (1 = linear)
])

connG = np.array([
    [1],           # Innovation numbers
    [5],           # Source node (should connect from ID 5)
    [7],           # Destination node (should connect to ID 7)
    [0.5],         # Weights
    [1]            # All enabled
])

# Test the function
Q, wMat = getNodeOrder(nodeG, connG)
print("Node order Q:", Q)
print("\nWeight Matrix:")
print(wMat)

IndexError: index 5 is out of bounds for axis 0 with size 3

In [25]:
conn = np.copy(connG)
node = np.copy(nodeG)
nIns = len(node[0,node[1,:] == 1]) + len(node[0,node[1,:] == 4])
nOuts = len(node[0,node[1,:] == 2])

# Create connection and initial weight matrices
conn[3,conn[4,:]==0] = np.nan # disabled but still connected
src  = conn[1,:].astype(int)
dest = conn[2,:].astype(int)

# A huge assumption here is Node is ordered by input, bias, output, hidden ... 
reordered_index = np.r_[node[0, node[1, :] ==1], node[0,node[1,:] == 4], node[0,node[1,:] == 2], node[0,node[1,:] == 3]]

# Vectorize Implementation 
src_mask = (src.reshape(-1, 1) == reordered_index.reshape(1, -1)) # (n_conn, n_node)
dest_mask = (dest.reshape(-1, 1) == reordered_index.reshape(1, -1))
src = (src_mask @ np.arange(len(reordered_index)).reshape(-1, 1)).flatten()  # Convert to 1D
dest = (dest_mask @ np.arange(len(reordered_index)).reshape(-1, 1)).flatten()  # Convert to 1D

# Create weight matrix (input, output, hidden)
wMat = np.zeros((np.shape(node)[1],np.shape(node)[1]))
wMat[src,dest] = conn[3,:] # assign weight to the connection

# Get connection matrix
connMat = wMat[nIns+nOuts:,nIns+nOuts:] # connection weights between hidden nodes
connMat[connMat!=0] = 1 # set non-zero weights to 1

# Topological Sort of Hidden Nodes
edge_in = np.sum(connMat,axis=0) # sum of edges ending with each node
Q = np.where(edge_in==0)[0]  # Start with nodes with no incoming connections
for i in range(len(connMat)):
    if (len(Q) == 0) or (i >= len(Q)):
        Q = []
        # return False, False # Cycle found, can't sort
    edge_out = connMat[Q[i],:]
    edge_in  = edge_in - edge_out # Remove nodes' conns from total
    nextNodes = np.setdiff1d(np.where(edge_in==0)[0], Q)
    Q = np.hstack((Q,nextNodes))

    if sum(edge_in) == 0:
        break

# Add In and outs back and reorder wMat according to sort
Q += nIns+nOuts # add index to give space for input and output nodes 

Q = np.r_[np.arange(nIns),                  # Input nodes use original node IDs
        Q,                              # Hidden nodes use indices
        np.arange(nIns,nIns+nOuts)]        # Output nodes use original node IDs


In [28]:
# wMat
connMat
# reordered_index

array([], shape=(0, 0), dtype=float64)

In [31]:
# Topological Sort of Hidden Nodes
edge_in = np.sum(connMat,axis=0) # sum of edges ending with each node
Q = np.where(edge_in==0)[0]  # Start with nodes with no incoming connections
for i in range(len(connMat)):
    if (len(Q) == 0) or (i >= len(Q)):
        Q = []
        # return False, False # Cycle found, can't sort
    edge_out = connMat[Q[i],:]
    edge_in  = edge_in - edge_out # Remove nodes' conns from total
    nextNodes = np.setdiff1d(np.where(edge_in==0)[0], Q)
    Q = np.hstack((Q,nextNodes))

    if sum(edge_in) == 0:
        break

# Add In and outs back and reorder wMat according to sort
Q += nIns+nOuts # add index to give space for input and output nodes 

Q = np.r_[np.arange(nIns),                  # Input nodes use original node IDs
        Q,                              # Hidden nodes use indices
        np.arange(nIns,nIns+nOuts)]        # Output nodes use original node IDs

wMat = wMat[np.ix_(Q,Q)]

In [32]:
wMat

array([[0. , 0. , 0.5],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])